In [1]:
"""
Asynchronous Advantage Actor Critic (A3C) with continuous action space, Reinforcement Learning.

The Pendulum example.

View more on my tutorial page: https://morvanzhou.github.io/tutorials/

Using:
tensorflow 1.0
gym 0.8.0
"""

import multiprocessing
import threading
import tensorflow as tf
import numpy as np
import gym
import os
import shutil
import matplotlib.pyplot as plt

GAME = 'BipedalWalker-v2'
#GAME = 'MountainCarContinuous-v0'
#GAME = 'Pendulum-v0'
OUTPUT_GRAPH = True
LOG_DIR = './log'
N_WORKERS = 2#multiprocessing.cpu_count()
MAX_EP_STEP = 800
MAX_GLOBAL_EP = 800000
GLOBAL_NET_SCOPE = 'Global_Net'
UPDATE_GLOBAL_ITER = 5
GAMMA = 0.95
ENTROPY_BETA = 0.1
LR_A = 0.00005    # learning rate for actor
LR_C = 0.001    # learning rate for critic
GLOBAL_RUNNING_R = []
GLOBAL_EP = 0

env = gym.make(GAME)

N_S = env.observation_space.shape[0]
N_A = env.action_space.shape[0]
A_BOUND = [env.action_space.low, env.action_space.high]


class ACNet(object):
    def __init__(self, scope, globalAC=None):

        if scope == GLOBAL_NET_SCOPE:   # get global network
            with tf.variable_scope(scope):
                self.s = tf.placeholder(tf.float32, [None, N_S], 'S')
                self._build_net()
                self.a_params = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope=scope + '/actor')
                self.c_params = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope=scope + '/critic')
        else:   # local net, calculate losses
            with tf.variable_scope(scope):
                self.s = tf.placeholder(tf.float32, [None, N_S], 'S')
                self.a_his = tf.placeholder(tf.float32, [None, N_A], 'A')
                self.v_target = tf.placeholder(tf.float32, [None, 1], 'Vtarget')

                mu, sigma, self.v = self._build_net()

                td = tf.subtract(self.v_target, self.v, name='TD_error')
                self.td = td
                with tf.name_scope('c_loss'):
                    self.c_loss = tf.reduce_mean(tf.square(td))

                with tf.name_scope('wrap_a_out'):
                    mu, sigma = mu * A_BOUND[1], sigma + 1e-10

                normal_dist = tf.contrib.distributions.Normal(mu, sigma)

                with tf.name_scope('a_loss'):
                    log_prob = normal_dist.log_prob(self.a_his)
                    exp_v = log_prob * td
                    entropy = normal_dist.entropy()  # encourage exploration
                    self.exp_v = ENTROPY_BETA * entropy + exp_v
                    self.a_loss = tf.reduce_sum(-self.exp_v)

                with tf.name_scope('choose_a'):  # use local params to choose action
                    self.A = tf.clip_by_value(tf.squeeze(normal_dist.sample(1), axis=0), A_BOUND[0], A_BOUND[1])
                with tf.name_scope('local_grad'):
                    self.a_params = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope=scope + '/actor')
                    self.c_params = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope=scope + '/critic')
                    self.a_grads = tf.gradients(self.a_loss, self.a_params)
                    self.c_grads = tf.gradients(self.c_loss, self.c_params)

            with tf.name_scope('sync'):
                with tf.name_scope('pull'):
                    self.pull_a_params_op = [l_p.assign(g_p) for l_p, g_p in zip(self.a_params, globalAC.a_params)]
                    self.pull_c_params_op = [l_p.assign(g_p) for l_p, g_p in zip(self.c_params, globalAC.c_params)]
                with tf.name_scope('push'):
                    self.update_a_op = OPT_A.apply_gradients(zip(self.a_grads, globalAC.a_params))
                    self.update_c_op = OPT_C.apply_gradients(zip(self.c_grads, globalAC.c_params))

    def _build_net(self ):
        w_init = tf.random_normal_initializer(0., .1)
        with tf.variable_scope('actor'):
            l_a_0 = tf.layers.dense(self.s, 512, tf.nn.relu6, kernel_initializer=w_init, name='la0')
            #l_a_1 = tf.layers.dense(l_a_0, 512, tf.nn.relu6, kernel_initializer=w_init, name='la1')
            mu = tf.layers.dense(l_a_0, N_A, tf.nn.tanh, kernel_initializer=w_init, name='mu')
            sigma = tf.layers.dense(l_a_0, N_A, tf.nn.softplus, kernel_initializer=w_init, name='sigma')
        with tf.variable_scope('critic'):
            l_c0 = tf.layers.dense(self.s, 256, tf.nn.relu6, kernel_initializer=w_init, name='lc0')
            #l_c1 = tf.layers.dense(l_c0, 256, tf.nn.relu6, kernel_initializer=w_init, name='lc1')
            v = tf.layers.dense(l_c0, 1, kernel_initializer=w_init, name='v')  # state value
        return mu, sigma, v
        

    def update_global(self, feed_dict):  # run by a local
        SESS.run([self.update_a_op, self.update_c_op], feed_dict)  # local grads applies to global net

    def pull_global(self):  # run by a local
        SESS.run([self.pull_a_params_op, self.pull_c_params_op])

    def choose_action(self, s):  # run by a local
        s = s[np.newaxis, :]
        return SESS.run(self.A, {self.s: s})[0]


class Worker(object):
    def __init__(self, name, globalAC):
        self.env = gym.make(GAME).unwrapped
        self.name = name
        self.AC = ACNet(name, globalAC)

    def work(self):
        global GLOBAL_RUNNING_R, GLOBAL_EP
        total_step = 1
        buffer_s, buffer_a, buffer_r = [], [], []
        
        while not COORD.should_stop() and GLOBAL_EP < MAX_GLOBAL_EP:
            s = self.env.reset()
            #add = np.linspace(0.5,0.3,1000)
            #step_ = 0
            ep_r = 0
            while True:
            #for ep_t in range(MAX_EP_STEP):
                if self.name == 'W_0':
                    self.env.render()
                r = 0                
                a = self.AC.choose_action(s)
                for _ in range(5):
                    if self.name == 'W_0':
                        self.env.render()
                    s_, r_, done, info = self.env.step(a)
                    #if not done:
                    #    r += 0.3
                    if not done and r_ > 0:
                        r_ *= 10
                    elif r > -99:
                        r_ = 0
                    #elif done:
                    #    r_ = 0
                    r += r_
                    ep_r += r_
                    if done:
                        break
                #step_ += 1
                #done = True if ep_t == MAX_EP_STEP - 1 else False
                #r /= 1     # normalize reward

                #ep_r += r
                buffer_s.append(s)
                buffer_a.append(a)
                buffer_r.append(r)
                

                if total_step % UPDATE_GLOBAL_ITER == 0 or done:   # update global and assign to local net
                    buffer_v_target = []
                    if done:
                        v_s_ = 0   # terminal
                    else:
                        v_s_ = SESS.run(self.AC.v, {self.AC.s: s_[np.newaxis, :]})[0, 0]
                    buffer_v_target.append(v_s_)
                    #buffer_v_target = []
                    for r, ss in zip(buffer_r[:-1][::-1], buffer_s[1:][::-1]):    # reverse buffer r
                        v_s_ = r + GAMMA * SESS.run(self.AC.v, {self.AC.s: ss[np.newaxis, :]})[0, 0]
                        buffer_v_target.append(v_s_)
                    buffer_v_target.reverse()

                    buffer_s, buffer_a, buffer_v_target = np.vstack(buffer_s), np.vstack(buffer_a), np.vstack(buffer_v_target)
                    feed_dict = {
                        self.AC.s: buffer_s,
                        self.AC.a_his: buffer_a,
                        self.AC.v_target: buffer_v_target,
                    }
                    self.AC.update_global(feed_dict)
                    buffer_s, buffer_a, buffer_r = [], [], []
                    self.AC.pull_global()
                    

                s = s_
                total_step += 1
                if done:
                    #print(SESS.run(self.AC.td,feed_dict))
                    if len(GLOBAL_RUNNING_R) == 0:  # record running episode reward
                        GLOBAL_RUNNING_R.append(ep_r)
                    else:
                        GLOBAL_RUNNING_R.append(0.9 * GLOBAL_RUNNING_R[-1] + 0.1 * ep_r)
                    print(
                        self.name,
                        "Ep:", GLOBAL_EP,
                        "| Ep_r: %i" % GLOBAL_RUNNING_R[-1],
                          )
                    GLOBAL_EP += 1
                    break

if __name__ == "__main__":
    SESS = tf.Session()

    with tf.device("/cpu:0"):
        OPT_A = tf.train.RMSPropOptimizer(LR_A, name='RMSPropA')
        OPT_C = tf.train.RMSPropOptimizer(LR_C, name='RMSPropC')
        GLOBAL_AC = ACNet(GLOBAL_NET_SCOPE)  # we only need its params
        workers = []
        # Create worker
        for i in range(N_WORKERS):
            i_name = 'W_%i' % i   # worker name
            workers.append(Worker(i_name, GLOBAL_AC))

    COORD = tf.train.Coordinator()
    SESS.run(tf.global_variables_initializer())

    if OUTPUT_GRAPH:
        if os.path.exists(LOG_DIR):
            shutil.rmtree(LOG_DIR)
        tf.summary.FileWriter(LOG_DIR, SESS.graph)

    worker_threads = []
    for worker in workers:
        job = lambda: worker.work()
        t = threading.Thread(target=job)
        t.start()
        worker_threads.append(t)
    COORD.join(worker_threads)

    plt.plot(np.arange(len(GLOBAL_RUNNING_R)), GLOBAL_RUNNING_R)
    plt.xlabel('step')
    plt.ylabel('Total moving reward')
    plt.show()



[2017-06-11 02:05:02,134] Making new env: BipedalWalker-v2
[2017-06-11 02:05:02,502] Making new env: BipedalWalker-v2
[2017-06-11 02:05:03,785] Making new env: BipedalWalker-v2


W_1 Ep: 0 | Ep_r: 634
W_1 Ep: 1 | Ep_r: 592
W_1 Ep: 2 | Ep_r: 534
W_1 Ep: 3 | Ep_r: 482
W_1 Ep: 4 | Ep_r: 455
W_1 Ep: 5 | Ep_r: 410
W_1 Ep: 6 | Ep_r: 370
W_1 Ep: 7 | Ep_r: 335
W_1 Ep: 8 | Ep_r: 302
W_1 Ep: 9 | Ep_r: 272
W_1 Ep: 10 | Ep_r: 244
W_1 Ep: 11 | Ep_r: 232
W_1 Ep: 12 | Ep_r: 223
W_1 Ep: 13 | Ep_r: 200
W_1 Ep: 14 | Ep_r: 180
W_1 Ep: 15 | Ep_r: 162
W_1 Ep: 16 | Ep_r: 146
W_1 Ep: 17 | Ep_r: 150
W_1 Ep: 18 | Ep_r: 135
W_1 Ep: 19 | Ep_r: 122
W_1 Ep: 20 | Ep_r: 110
W_1 Ep: 21 | Ep_r: 99
W_1 Ep: 22 | Ep_r: 91
W_1 Ep: 23 | Ep_r: 82
W_1 Ep: 24 | Ep_r: 74
W_1 Ep: 25 | Ep_r: 68
W_1 Ep: 26 | Ep_r: 62
W_1 Ep: 27 | Ep_r: 57
W_1 Ep: 28 | Ep_r: 56
W_1 Ep: 29 | Ep_r: 51
W_1 Ep: 30 | Ep_r: 47
W_1 Ep: 31 | Ep_r: 43
W_1 Ep: 32 | Ep_r: 38
W_1 Ep: 33 | Ep_r: 34
W_1 Ep: 34 | Ep_r: 32
W_1 Ep: 35 | Ep_r: 31
W_1 Ep: 36 | Ep_r: 28
W_1 Ep: 37 | Ep_r: 30
W_1 Ep: 38 | Ep_r: 28
W_1 Ep: 39 | Ep_r: 29
W_1 Ep: 40 | Ep_r: 26
W_1 Ep: 41 | Ep_r: 43
W_1 Ep: 42 | Ep_r: 39
W_1 Ep: 43 | Ep_r: 36
W_1 Ep: 44 | Ep_r: 36

Exception in thread Thread-5:
Traceback (most recent call last):
  File "/home/damody/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/damody/anaconda3/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-1-1a0a258f6403>", line 232, in <lambda>
    job = lambda: worker.work()
  File "<ipython-input-1-1a0a258f6403>", line 145, in work
    self.env.render()
  File "/home/damody/gym/gym/core.py", line 153, in render
    return self._render(mode=mode, close=close)
  File "/home/damody/gym/gym/envs/box2d/bipedal_walker.py", line 487, in _render
    return self.viewer.render(return_rgb_array = mode=='rgb_array')
  File "/home/damody/gym/gym/envs/classic_control/rendering.py", line 84, in render
    self.window.dispatch_events()
  File "/home/damody/anaconda3/lib/python3.6/site-packages/pyglet/window/xlib/__init__.py", line 853, in dispatch_events
    0x1ffffff, byref(e)):
ctypes.Arg

W_1 Ep: 1357 | Ep_r: 3395
W_1 Ep: 1358 | Ep_r: 3468
W_1 Ep: 1359 | Ep_r: 3270
W_1 Ep: 1360 | Ep_r: 3362
W_1 Ep: 1361 | Ep_r: 3421
W_1 Ep: 1362 | Ep_r: 3385
W_1 Ep: 1363 | Ep_r: 3464
W_1 Ep: 1364 | Ep_r: 3511
W_1 Ep: 1365 | Ep_r: 3560
W_1 Ep: 1366 | Ep_r: 3617
W_1 Ep: 1367 | Ep_r: 3678
W_1 Ep: 1368 | Ep_r: 3721
W_1 Ep: 1369 | Ep_r: 3571


KeyboardInterrupt: 